<a href="https://colab.research.google.com/github/PatrickRuan/AI_2025/blob/main/07_ollama_web_in_memory_asyn_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab_Ollama_Flask_ngrok_Demo.ipynb

# 安裝必要套件
!pip install --upgrade openai flask pyngrok

# 啟動 Ollama (注意：Colab 通常無法直接安裝與啟動 Ollama，需要自己有伺服器)
# 如果本地安裝且能啟動 Ollama，則用以下命令：
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve > ollama.log 2>&1 &
!sleep 5
!ollama pull gemma3:4b
from IPython.display import clear_output
clear_output(wait=True)
print("✅ 模型下載完成！")

✅ 模型下載完成！


In [ ]:

# 設定 Ngrok 連線
from pyngrok import ngrok, conf
from getpass import getpass

# 輸入 ngrok authtoken
auth_token = getpass("請輸入你的 ngrok authtoken：")
conf.get_default().auth_token = auth_token

# 開啟 5080 埠的 ngrok 隧道
public_url = ngrok.connect(5080)
print(f"\u2728 Ngrok 公開網址: {public_url}")

# 建立 Flask + Local Ollama 服務
from flask import Flask, request, jsonify, render_template_string
from openai import OpenAI
import threading
import os
import logging

# 初始化 Flask
app = Flask(__name__)

# 設置日誌
logging.basicConfig(level=logging.DEBUG)

# 連接本地 Ollama API
client = OpenAI(
    base_url="http://localhost:11434/v1",  # 本地 Ollama 服務
    api_key="test-key"  # 隨便填
)

# 模型名稱
model = "gemma3:4b"

# 對話記憶
conversation_memory = []

# HTML 模板
html_template = f"""
<!DOCTYPE html>
<html lang="zh-Hant">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Ollama Chatbot</title>
    <style>
        body {{ font-family: Arial, sans-serif; padding: 20px; }}
        .chat-box {{ width: 100%; max-width: 600px; margin: auto; }}
        .messages {{ border: 1px solid #ccc; padding: 10px; height: 400px; overflow-y: scroll; }}
        .input-box {{ display: flex; margin-top: 10px; }}
        input[type='text'] {{ flex: 1; padding: 10px; }}
        button {{ padding: 10px; background: green; color: white; border: none; cursor: pointer; }}
    </style>
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
</head>
<body>
    <div class="chat-box">
        <h2>Ollama Chatbot ({model})</h2>
        <div class="messages" id="messages"></div>
        <div class="input-box">
            <input type="text" id="userMessage" placeholder="輸入訊息..." onkeydown="handleKeyPress(event)">
            <button onclick="sendMessage()">送出</button>
        </div>
    </div>
<script>
function appendMessage(role, content) {{
    const messages = document.getElementById('messages');
    const div = document.createElement('div');
    div.innerHTML = '<b>' + role + ':</b> ' + marked.parse(content);
    messages.appendChild(div);
    messages.scrollTop = messages.scrollHeight;
}}

function sendMessage() {{
    const userMessage = document.getElementById('userMessage').value;
    if (!userMessage.trim()) return;
    appendMessage('User', userMessage);
    document.getElementById('userMessage').value = '';
    fetch('/chat', {{
        method: 'POST',
        headers: {{ 'Content-Type': 'application/json' }},
        body: JSON.stringify({{message: userMessage}})
    }}).then(response => response.json())
      .then(data => appendMessage('Assistant', data.reply))
      .catch(error => console.error('錯誤:', error));
}}

function handleKeyPress(event) {{
    if (event.key === 'Enter') {{
        sendMessage();
    }}
}}
</script>
</body>
</html>
"""

# 首頁
@app.route('/')
def index():
    return render_template_string(html_template)

# 對話 API
@app.route('/chat', methods=['POST'])
def chat():
    data = request.get_json()
    user_message = data.get('message', '')
    if not user_message:
        return jsonify({"reply": "Error: no input message."})

    conversation_memory.append({"role": "user", "content": user_message})

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "總是用繁體中文回答。"},
                *conversation_memory
            ]
        )
        reply = response.choices[0].message.content
        conversation_memory.append({"role": "assistant", "content": reply})
        return jsonify({"reply": reply})
    except Exception as e:
        return jsonify({"reply": f"Error: {str(e)}"})

# 啟動 Flask
app.run(host='0.0.0.0', port=5080)

請輸入你的 ngrok authtoken：··········
✨ Ngrok 公開網址: NgrokTunnel: "https://f8fa-34-143-217-146.ngrok-free.app" -> "http://localhost:5080"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5080
 * Running on http://172.28.0.12:5080
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/Apr/2025 11:58:20] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2025 11:58:20] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2025 11:58:37] "POST /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2025 11:58:37] "POST /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2025 11:58:49] "POST /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2025 11:59:42] "POST /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2025 11:59:45] "POST /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2025 12:00:20] "POST /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2025 12:00:50] "POST /chat HTTP/1.1" 200 -
